<a href="https://colab.research.google.com/github/FabienMiguel/NLP-Fellowship/blob/main/Fabien_First_Model_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First Model Assignment

The aim of this assignment is to make sure you understand the foundations of model training. We have covered traditional ML and also simple NN. You task is

to write code for training traditional ML model that gives the highest accuracy
Code for NN model that gives the highest accuracy
This we will consider
The code works
The hyperparameter used for fine tuning - epoch only is not enough
The highest accuracy
Bonus points for being creative with preprocessing, tokenization and creation of vectors
Submissions
Notebook with code
2 models
Deadline
Monday 28th at 5pm

ASSIGNMENT ANSWERS

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import pandas as pd

full_dataset = pd.read_csv('/content/drive/MyDrive/NLP Fellowship/50k_imdb_movie_reviews (1).csv')
full_dataset

,review,sentiment,set
0,I went and saw this movie last night after bei...,1,test
1,Actor turned director Bill Paxton follows up h...,1,test
2,As a recreational golfer with some knowledge o...,1,test
3,"I saw this film in a sneak preview, and it is ...",1,test
4,Bill Paxton has taken the true story of the 19...,1,test
...,...,...,...
49995,"Towards the end of the movie, I felt it was to...",0,train
49996,This is the kind of movie that my enemies cont...,0,train
49997,I saw 'Descent' last night at the Stockholm Fi...,0,train
49998,Some films that you pick up for a pound turn o...,0,train


In [7]:
!pip install torch==1.8.0 torchtext==0.9.0 #compatibility

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 735.5 MB 14 kB/s 
     |████████████████████████████████| 7.1 MB 17.6 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.0 which is incompatible.


In [8]:
import pandas as pd
import re
import torch
from torchtext.legacy import data
from torchtext.legacy.data import Dataset, Example
from torchtext.legacy.data import BucketIterator
import torch.nn as nn
import torch.nn.functional as F

In [9]:
full_dataset['review'].describe()

count                                                 50000
unique                                                49582
top       Loved today's show!!! It was a variety and not...
freq                                                      5
Name: review, dtype: object

In [10]:
full_dataset = full_dataset.drop_duplicates(subset=['review'])

In [11]:
full_dataset = full_dataset.drop_duplicates(subset=['review'])
train_dataset = full_dataset[(full_dataset['set'] == 'train')][['review','sentiment']]
test_dataset = full_dataset[(full_dataset['set'] == 'test')][['review','sentiment']]

In [12]:
# full_dataset
def preprocessing(texts):
  cleaned_text = []
  for text in texts:
    text = text.lower()
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    text = emoji_pattern.sub(r'', text)
    text = url_pattern.sub(r'', text)
    text = html_pattern.sub(r'', text)
    text = re.sub(r"[^\w\d'\s]+", ' ', text)
    cleaned_text.append(text)

  return cleaned_text

In [13]:
SEED = 42
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True # Check this
max_document_length = 300 #hyperparameter

TEXT = data.Field(lower=True, include_lengths=True,  tokenize='spacy',preprocessing=preprocessing,batch_first=True,  fix_length=max_document_length)
LABEL = data.Field(sequential=False, use_vocab=False)

class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )

/usr/local/lib/python3.7/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [14]:

torch_valid_dataset, torch_test_dataset = DataFrameDataset(
    df=test_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
).split() 

torch_train_dataset = DataFrameDataset(
    df=train_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
)

In [15]:

max_size = 30000 #hyperparameter
TEXT.build_vocab(torch_train_dataset, max_size=max_size,vectors='fasttext.simple.300d')
vocab_size = len(TEXT.vocab)

BATCH_SIZE = 64 #hyperparameter
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (torch_train_dataset, torch_valid_dataset, torch_test_dataset), 
    batch_size = BATCH_SIZE ,
    sort_key=lambda x: len(x.review),
    sort_within_batch=True)

.vector_cache/wiki.simple.vec: 293MB [00:35, 8.35MB/s]                           
100%|██████████| 111051/111051 [00:08<00:00, 12638.13it/s]


**accuracy function to be accessed by both models**

GLOBAL HYPERPARAMETERS

In [16]:
lr = 1e-3
batch_size = 64
dropout_keep_prob = 0.3
embedding_size = 300
max_document_length = 300 # each sentence has until 100 words
vocab_size = len(TEXT.vocab)
dev_size = 0.8 # split percentage to train\validation data
max_size = 30000 # maximum vocabulary size
seed = 30
num_classes = 2

num_epochs = 15
hidden_size = 256
hidden_size1 = 300
hidden_size2 = 128
hidden_size3 = 64

In [17]:
def accuracy(probs, target):
  winners = probs.argmax(dim=1)
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

**LR neural networks**

In [18]:
# full_dataset
class LR(nn.Module):
    def __init__(self, input_size, hidden_size,hidden_size2,hidden_size3, num_classes):
        super(LR, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3) 
        self.fc4 = nn.Linear(hidden_size3, num_classes)

    def forward(self, text):
        text = text.float() # dense layer deals just with float type data
        x = self.fc1(text) #(m x n) with (n x p)
        x = self.relu(x)
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        
        preds = self.fc4(x) # crossentropyloss handles the softmax
        # preds = F.softmax(preds,1) # nn.softmax
        return preds

In [19]:
to_train = True
LRmodel = LR(max_document_length, hidden_size,hidden_size2,hidden_size3, num_classes)

In [20]:
LRbest_valid_loss = float('inf')
LRoptimizer = torch.optim.Adam(LRmodel.parameters(), lr=lr)

LRloss_func = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
  LRtrain_epoch_loss = 0
  LRtrain_epoch_acc = 0
  for batch in train_iterator:
      LRoptimizer.zero_grad()
      # retrieve text and no. of words
      text, text_lengths = batch.review

      #feedforward
      # model.to(device)
      LRpredictions = LRmodel(text).squeeze(1)
      
      
      loss = LRloss_func(LRpredictions, batch.sentiment)

      acc = accuracy(LRpredictions, batch.sentiment)

      # perform backpropagation
      loss.backward()

      LRoptimizer.step()

      LRtrain_epoch_loss += loss.item()
      LRtrain_epoch_acc += acc.item()

  

  LRvalid_epoch_loss = 0
  LRvalid_epoch_acc = 0

  LRmodel.eval()

  with torch.no_grad():
      for batch in valid_iterator:
          text, text_lengths = batch.review

          LRpredictions = LRmodel(text).squeeze(1)

          loss = LRloss_func(LRpredictions, batch.sentiment)

          acc = accuracy(LRpredictions, batch.sentiment)

          LRvalid_epoch_loss += loss.item()
          LRvalid_epoch_acc += acc.item()

   

  if LRvalid_epoch_loss < LRbest_valid_loss:
            LRbest_valid_loss = LRvalid_epoch_loss
            torch.save(LRmodel.state_dict(), 'LRsaved_weights'+'_LRlinear.pt')

  print(f'\tTrain Loss: {LRtrain_epoch_loss / len(train_iterator):.3f} | Train Acc: {LRtrain_epoch_acc  / len(train_iterator)* 100:.2f}%')
  print(f'\t Val. Loss: {LRvalid_epoch_loss / len(valid_iterator):.3f} |  Val. Acc: {LRvalid_epoch_acc / len(valid_iterator) * 100:.2f}%')

	Train Loss: 2.886 | Train Acc: 49.58%
	 Val. Loss: 0.697 |  Val. Acc: 50.48%
	Train Loss: 0.693 | Train Acc: 50.25%
	 Val. Loss: 0.699 |  Val. Acc: 50.05%
	Train Loss: 0.692 | Train Acc: 51.08%
	 Val. Loss: 0.697 |  Val. Acc: 50.23%
	Train Loss: 0.689 | Train Acc: 50.88%
	 Val. Loss: 0.695 |  Val. Acc: 50.39%
	Train Loss: 0.687 | Train Acc: 52.03%
	 Val. Loss: 0.705 |  Val. Acc: 50.07%
	Train Loss: 0.685 | Train Acc: 52.01%
	 Val. Loss: 0.696 |  Val. Acc: 50.35%
	Train Loss: 0.683 | Train Acc: 52.32%
	 Val. Loss: 0.699 |  Val. Acc: 50.42%
	Train Loss: 0.685 | Train Acc: 52.05%
	 Val. Loss: 0.703 |  Val. Acc: 50.47%
	Train Loss: 0.685 | Train Acc: 52.04%
	 Val. Loss: 0.694 |  Val. Acc: 50.07%
	Train Loss: 0.682 | Train Acc: 52.04%
	 Val. Loss: 0.705 |  Val. Acc: 50.54%
	Train Loss: 0.685 | Train Acc: 52.18%
	 Val. Loss: 0.703 |  Val. Acc: 50.06%
	Train Loss: 0.684 | Train Acc: 52.39%
	 Val. Loss: 0.718 |  Val. Acc: 50.25%
	Train Loss: 0.682 | Train Acc: 53.03%
	 Val. Loss: 0.731 |  Val

%%%%%%%%%%%%%%%%%%%%%%% multilayer percetron neural networks %%%%%%%%%%%%%%%%%%%%%%% 

In [21]:
class MLP(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size2, hidden_size3, hidden_size4, output_dim, dropout, max_document_length):
        super().__init__()
        # embedding and convolution layers
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(embed_size*max_document_length, hidden_size2)  # dense layer
        self.fc2 = nn.Linear(hidden_size2, hidden_size3)  # dense layer
        self.fc3 = nn.Linear(hidden_size3, hidden_size4)  # dense layer
        self.fc4 = nn.Linear(hidden_size4, output_dim)  # dense layer

    def forward(self, text):
         # text shape = (batch_size, num_sequences)
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
        
        x = embedded.view(embedded.shape[0], -1)  # x = Flatten()(x)
        #embedded = embedded.unsqueeze(1) # fc gets 4 dimension
        
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        preds = self.fc4(x)
        # preds = F.softmax(preds, 1)
        return preds

In [22]:

to_train = True
# model = LR(max_document_length, hidden_size,hidden_size2,hidden_size3, num_classes)
model = MLP(vocab_size, embedding_size, hidden_size1, hidden_size2, hidden_size3,  num_classes, dropout_keep_prob, max_document_length)

In [ ]:
best_valid_loss = float('inf')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

loss_func = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
  train_epoch_loss = 0
  train_epoch_acc = 0
  for batch in train_iterator:
      optimizer.zero_grad()
      # retrieve text and no. of words
      text, text_lengths = batch.review

      #feedforward
      # model.to(device)
      predictions = model(text).squeeze(1)
      
      
      loss = loss_func(predictions, batch.sentiment)

      acc = accuracy(predictions, batch.sentiment)

      # perform backpropagation
      loss.backward()

      optimizer.step()

      train_epoch_loss += loss.item()
      train_epoch_acc += acc.item()

  

  valid_epoch_loss = 0
  valid_epoch_acc = 0

  model.eval()

  with torch.no_grad():
      for batch in valid_iterator:
          text, text_lengths = batch.review

          predictions = model(text).squeeze(1)

          loss = loss_func(predictions, batch.sentiment)

          acc = accuracy(predictions, batch.sentiment)

          valid_epoch_loss += loss.item()
          valid_epoch_acc += acc.item()

   

  if valid_epoch_loss < best_valid_loss:
            best_valid_loss = valid_epoch_loss
            torch.save(model.state_dict(), 'saved_weights'+'_linear.pt')

  print(f'\tTrain Loss: {train_epoch_loss / len(train_iterator):.3f} | Train Acc: {train_epoch_acc  / len(train_iterator)* 100:.2f}%')
  print(f'\t Val. Loss: {valid_epoch_loss / len(valid_iterator):.3f} |  Val. Acc: {valid_epoch_acc / len(valid_iterator) * 100:.2f}%')

	Train Loss: 0.763 | Train Acc: 50.19%
	 Val. Loss: 0.693 |  Val. Acc: 51.35%
	Train Loss: 0.676 | Train Acc: 55.57%
	 Val. Loss: 0.692 |  Val. Acc: 53.53%
	Train Loss: 0.533 | Train Acc: 65.58%
	 Val. Loss: 0.715 |  Val. Acc: 54.85%
	Train Loss: 0.481 | Train Acc: 67.84%
	 Val. Loss: 0.895 |  Val. Acc: 53.18%
